In [18]:
from microfaune.detection import RNNDetector
import csv
import os
import glob
import pandas as pd
from microfaune import audio
import scipy.signal as scipy_signal
from IPython.display import clear_output
from shutil import copyfile

In [19]:
weightsPath = ""
XCDataPath = ""
column_names = ["Folder","Clip","Bird_Label","Global Score"]
df = pd.DataFrame(columns = column_names)
bird_detector = RNNDetector(weightsPath)
Normalized_Sample_Rate = 44100

In [20]:
dataList = []

In [23]:
list = os.listdir(XCDataPath) # dir is your directory path
num_filesXC = len(list)
countXC = 0
errCount = 0
repCountXC = 1
repListXC = []
# with open("DAXC.csv",mode='a') as dataset:
#         writer = csv.writer(dataset,delimiter=",")
#         writer.writerow(["Folder","Clip","Bird_Label","Global Score"])
for file in glob.glob(XCDataPath + "*.wav"):
    path_list = file.split("/")
    folder_name = path_list[len(path_list) - 2 ]
    clip_name = path_list[len(path_list) - 1 ]
    
    if "(1)" in clip_name:
        repCountXC += 1
        repListXC.append(clip_name)
        continue
    
    SAMPLE_RATE, SIGNAL = audio.load_wav(XCDataPath + clip_name)
    
    # downsample the audio if the sample rate > 44.1 kHz
    # Force everything into the human hearing range.
    # May consider reworking this function so that it upsamples as well
    if SAMPLE_RATE > Normalized_Sample_Rate:
        rate_ratio = Normalized_Sample_Rate / SAMPLE_RATE
        SIGNAL = scipy_signal.resample(
                SIGNAL, int(len(SIGNAL)*rate_ratio))
        SAMPLE_RATE = Normalized_Sample_Rate
        # resample produces unreadable float32 array so convert back
        #SIGNAL = np.asarray(SIGNAL, dtype=np.int16)

    #print(SIGNAL.shape)
    # convert stereo to mono if needed
    # Might want to compare to just taking the first set of data.
    if len(SIGNAL.shape) == 2:
        SIGNAL = SIGNAL.sum(axis=1) / 2
        
    try:
        microfaune_features = bird_detector.compute_features([SIGNAL])
        global_score, local_score = bird_detector.predict(microfaune_features)
        clear_output(wait=True)
        dataList.append([folder_name, clip_name,'y',global_score[0][0]])
        countXC += 1
        print(str(countXC) + "/" + str(num_filesXC))
        
    except:
        print(file + " Failed")
        errCount += 1
        continue
#     with open("DAXC.csv",mode='a') as dataset:
#         writer = csv.writer(dataset,delimiter=",")
#         writer.writerow([folder_name,clip_name,'y',global_score[0][0]])
print("Errors: " + str(errCount))

4774/4776
Errors: 1


In [24]:
nonBirdPath = ""
list = os.listdir(nonBirdPath) # dir is your directory path
num_files = len(list)
countNB = 0
errCount = 0
repCountNB = 0
repListNB = []
for file in glob.glob(nonBirdPath + "*.wav"):
    path_list = file.split("/")
    folder_name = path_list[len(path_list) - 2 ]
    clip_name = path_list[len(path_list) - 1 ]
    
    if "(1)" in clip_name:
        repCountNB += 1
        repListNB.append(clip_name)
        continue
    
    SAMPLE_RATE, SIGNAL = audio.load_wav(nonBirdPath + clip_name)
    
    # downsample the audio if the sample rate > 44.1 kHz
    # Force everything into the human hearing range.
    # May consider reworking this function so that it upsamples as well
    if SAMPLE_RATE > Normalized_Sample_Rate:
        rate_ratio = Normalized_Sample_Rate / SAMPLE_RATE
        SIGNAL = scipy_signal.resample(
                SIGNAL, int(len(SIGNAL)*rate_ratio))
        SAMPLE_RATE = Normalized_Sample_Rate
        # resample produces unreadable float32 array so convert back
        #SIGNAL = np.asarray(SIGNAL, dtype=np.int16)

    #print(SIGNAL.shape)
    # convert stereo to mono if needed
    # Might want to compare to just taking the first set of data.
    if len(SIGNAL.shape) == 2:
        SIGNAL = SIGNAL.sum(axis=1) / 2
    
    try:
        microfaune_features = bird_detector.compute_features([SIGNAL])
        global_score, local_score = bird_detector.predict(microfaune_features)
        clear_output(wait=True)
        dataList.append([folder_name,clip_name,'n',global_score[0][0]])
        countNB += 1
        print(str(countNB) + "/" + str(num_files))
        # There are more non bird files than bird present files so we balance them
        if (countNB >= countXC):
            break
    except:
        print(file + " Failed")
        errCount += 1
        continue
print("Errors: " + str(errCount))

4774/6448
Errors: 0


In [26]:
df = pd.DataFrame(dataList, columns = ["Folder","Clip","Bird_Label","Global Score"])

In [27]:
df

,Folder,Clip,Bird_Label,Global Score
0,XCSelection,Actitis-macularius-415533.wav,y,0.989233
1,XCSelection,Cacicus-cela-59783.wav,y,0.979663
2,XCSelection,Chiroxiphia-boliviana-30214.wav,y,0.977731
3,XCSelection,Mecocerculus-leucophrys-468875.wav,y,0.987400
4,XCSelection,Troglodytes-aedon-59451.wav,y,0.982911
...,...,...,...,...
9543,Audioset_nonBird,ENCPkF4K9zU_30.000.wav,n,0.211858
9544,Audioset_nonBird,c_c06qrLbo4_90.000.wav,n,0.035369
9545,Audioset_nonBird,x0fI4QXOz9M_30.000.wav,n,0.145443
9546,Audioset_nonBird,JtlBTAm3ImM_130.000.wav,n,0.270606


In [29]:
csvName = ""
df.to_csv(csvName)